In [ ]:
# BigQueryML Demo - Sept 2021 

Source: [Predicting customer propensity to buy by using BigQuery ML and AI Platform](https://cloud.google.com/architecture/predicting-customer-propensity-to-buy?hl=en)

Related: [Building a k-means clustering model for market segmentation by using BigQuery ML](https://cloud.google.com/architecture/building-k-means-clustering-model)

In [2]:
# Install libraries.
!pip install pandas-gbq
!pip install google-cloud-bigquery
!pip install google-cloud-bigquery-storage
# Needed to setup flex slots for flat-rate pricing
!pip install google-cloud-bigquery-reservation

# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

     |████████████████████████████████| 59 kB 3.9 MB/s eta 0:00:011


{'status': 'ok', 'restart': True}

In [1]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt

# used to display float format
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
client = bigquery.Client(project="justinjm-project-01")

## Create training dataset

In [3]:
query1 = '''
# select initial features and label to feed into your model
CREATE OR REPLACE TABLE bqml.propensity_data2 AS
  SELECT
    fullVisitorId,
    bounces,
    time_on_site,
    will_buy_on_return_visit
  FROM (
        # select features
        SELECT
          fullVisitorId,
          IFNULL(totals.bounces, 0) AS bounces,
          IFNULL(totals.timeOnSite, 0) AS time_on_site
        FROM
          `data-to-insights.ecommerce.web_analytics`
        WHERE
          totals.newVisits = 1
        AND date BETWEEN '20160801' # train on first 9 months of data
        AND '20170430'
       )
  JOIN (
        SELECT
          fullvisitorid,
          IF (
              COUNTIF (
                       totals.transactions > 0
                       AND totals.newVisits IS NULL
                      ) > 0,
              1,
              0
             ) AS will_buy_on_return_visit
        FROM
          `bigquery-public-data.google_analytics_sample.*`
        GROUP BY
          fullvisitorid
       )
  USING (fullVisitorId)
  ORDER BY time_on_site DESC
'''

# bq_view.view_query = query.format('justinjm-project-01')
# bq_view = client.create_table(bq_view)


NameError: name 'bq_view' is not defined

In [6]:
# Show a sample of GA360 data

ga360_query_df = f'''
SELECT * FROM `bqml_demo.propensity_data`
ORDER BY fullVisitorID
LIMIT 5
'''

job_config = bigquery.QueryJobConfig()

# Start the query
query_job = client.query(ga360_query_df, job_config=job_config) #API Request
df_ga360 = query_job.result()
df_ga360 = df_ga360.to_dataframe()

df_ga360

,fullVisitorId,bounces,time_on_site,will_buy_on_return_visit
0,0000010278554503158,0,194,0
1,0000020424342248747,0,182,0
2,0000020424342248747,0,297,0
3,0000027376579751715,0,49,0
4,0000039460501403861,0,99,0
